# Predicción Automática Días de Pago de Factura

<a id="1"></a>
## 1. Preparación del entorno

Preparamos las heramientas y librerías para ajustar los modelos

### Cargar Librerías

Librerías pyhton para manejo de los datasets proporcionados.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from seaborn import lmplot
from matplotlib import pyplot as plt
import xgboost as xgb
from xgboost import cv
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score, LeaveOneOut
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
import sklearn.preprocessing as preprocessing
from sklearn.metrics import accuracy_score, r2_score, mean_squared_error, confusion_matrix, classification_report
import datetime as dt
from datetime import date, datetime
#import eli5
#from eli5.sklearn import PermutationImportance
from statistics import mean
from sklearn import datasets, metrics
from sklearn.utils import class_weight

<a id="2"></a>
## 2. Cargar y explorar los datos

In [ ]:
datos_facturas = pd.read_csv('datasets/facturas_3.csv',sep=',',low_memory=False, index_col=0)
datos_facturas = datos_facturas.drop(['id_c_documento', 'numero_documento','importe_nota_credito', 'importe_descuento',
                                      'id_c_tipo_documento', 'importe_pago', 'id_c_tipo_cliente', 'conciliada', 
                                      'fecha_envia_valija', 'fecha_inserta_fisica', 'id_user_crea', 'clave_sap',
                                      'contrato_documento', 'observacion_documento'], axis=1)
shape = datos_facturas.shape
print(shape)
datos_facturas

### Preparación del Dataset para el modelo XGBoost

In [ ]:
datos_facturas.columns.values

Variables incluidas en nuestro modelo:
1. rfc_cliente  
2. rfc_deudor  
3. id_division_deudor  
4. tipo_cliente  
5. id_division_documento
6. giro_cliente
7. fecha_reprogramacion 
8. fecha_deposito  
9. fecha_pago  
10. num_zona  
11. pedido  
12. id_c_moneda  
13. id_c_deudor  
14. id_c_cliente  
15. fecha_documento  
16. importe_documento  
17. saldo_documento  
18. id_c_ruta  
19. fecha_recibida_aeesa  
20. contra_recibo  
21. id_c_anomalia  
22. fecha_inserta  
23. status  
24. id_c_tipo_localidad  
25. dias_credito
26. fecha_revision
27. fecha_anomalia
28. num_visitas

In [ ]:
dates = datos_facturas[['fecha_documento', 'fecha_recibida_aeesa', 'fecha_pago', 'fecha_reprogramacion', 'fecha_revision',
                       'fecha_inserta']]

In [ ]:
for i in dates:
    datos_facturas[i] = pd.to_datetime(datos_facturas[i], format = '%Y-%m-%d')

In [ ]:
datos_facturas.dtypes

<a id="3"></a>
## 3. Ajuste fino del modelo

In [ ]:
categoricals = datos_facturas[['rfc_cliente', 'rfc_deudor', 'id_division_deudor', 'tipo_cliente', 'giro_cliente',
                               'id_division_documento', 'num_zona', 'pedido', 'id_c_deudor', 'id_c_cliente', 'contra_recibo',
                               'id_c_anomalia', 'status', 'id_c_moneda', 'id_c_tipo_localidad', 'id_c_ruta']]
numericals = datos_facturas[['importe_documento', 'saldo_documento', 'dias_credito', 'num_visitas']]
dates = datos_facturas[['fecha_documento', 'fecha_recibida_aeesa', 'fecha_pago', 'fecha_reprogramacion', 'fecha_revision',
                       'fecha_inserta', 'fecha_deposito', 'fecha_anomalia']]

In [ ]:
datos_facturas.dtypes

In [ ]:
cat = categoricals.columns.values
encoder = OrdinalEncoder()
def cat_num (categoricals):
    for i in cat:
        encoder.fit(categoricals[[i]])
        categoricals[i] = encoder.transform(categoricals[[i]])
    return categoricals

In [ ]:
categoricals = cat_num(categoricals)

In [ ]:
for i in dates:
    dates[i] = pd.to_datetime(dates[i], format = '%Y-%m-%d')

In [ ]:
datos_facturas_xgb = pd.merge(pd.merge(categoricals,numericals, left_index=True, right_index=True),
                              dates, left_index=True, right_index=True)
datos_facturas_xgb.dtypes

In [ ]:
datos_facturas_xgb.shape

A continuación pasaremos todos los importes y saldos de las facturas a pesos mexicanos (0), esto es necesario puesto que hay facturas en dólares estadounidenses (1).

In [ ]:
tipodecambio = pd.read_excel('datasets/Tipodecambio.xlsx', index_col=0)
tipodecambio.head()

In [ ]:
df1 = datos_facturas_xgb[datos_facturas_xgb['id_c_moneda']==1]
df1.shape

In [ ]:
datos_facturas_xgb.drop(datos_facturas_xgb[datos_facturas_xgb['id_c_moneda']==1].index, inplace=True)

In [ ]:
print(tipodecambio['fecha_cambio'].max())

In [ ]:
df2 = df1[df1['fecha_documento']>'2023-01-09']
df2.shape

Como acabamos de poder ver, se pierden 16 facturas debido a que su fecha de emisión es posterior a la última fecha registrada por los registros del tipo de cambio que tenemos.

In [ ]:
tipodecambio.rename(columns={'fecha_cambio': 'fecha_documento'}, inplace=True)
df1 = pd.merge(df1, tipodecambio, on = 'fecha_documento')
df1.shape

In [ ]:
def cambio_usd_pesos(df):
    for i in range(0, len(df)):
        if df.loc[i, 'id_c_moneda']==1:
                df.loc[i, 'importe_documento'] = df.loc[i, 'importe_documento']*df.loc[i, 'tipo_cambio']
                df.loc[i, 'saldo_documento'] = df.loc[i, 'saldo_documento']*df.loc[i, 'tipo_cambio']
    return df

In [ ]:
cambio_usd_pesos(df1)
datos_facturas_xgb = pd.concat([datos_facturas_xgb, df1])

Hemos perdido 16 registros de facturas (que antes de aplicar este cambio para incluir las facturas que estuviesen en base dólares estadounidense no estaban de por si incluidas en el modelo) y hemos ganado 1063 facturas para nuestro modelo, por lo que consideraremos esta la mejor estrategia.

In [ ]:
datos_facturas_xgb = datos_facturas_xgb.drop(['tipo_cambio'], axis=1)
datos_facturas_xgb.head()

In [ ]:
datos_facturas_xgb.shape

**Problemas con esta parte:**
- El registro del tipo de cambio USD-pesos limita las facturas a entrenar de nuestro modelo, habría que tener una variable que se vaya actualizando con el tipo de cambio medio diario para que cada vez que se realice el entrenamiento del modelo no se pierdan registros de facturas.

[Volver al inicio del sección](#3)

[Volver al Índice](#contenido)

<a id="4"></a>
## 4. Estrategia

## ¿Cuánto se tarda en pagar?
Este modelo tratará de responder a esta pregunta. Nuestra estrategia será la siguiente:
1. Eliminamos las facturas que estén vacías en las columnas de las variables fecha_documento y fecha_pago, sin estos datos jamás podremos predecir lo que se tarda en pagar puesto que es el resultado de la resta de estos dos valores.
2. Crearemos nuestra variable objetivo que nombraremos como target, esta es el resultado de la operación fecha_pago menos fecha_documento.
3. Por último, para introducir la información que nos otorgan las fechas en nuestro modelo XGBoost, tendremos que partir de una fecha de referencia, en este caso será fecha_documento. Dicha fecha se la restaremos al resto de fechas y nos quedarán unas nuevas variables (nombradas de la misma manera) pero en formato de días. Para finalizar, eliminaremos la columna fecha_documento, dado que ya no nos aporta ninguna información.

In [ ]:
datos_facturas_xgb = datos_facturas_xgb[datos_facturas_xgb['fecha_documento'].notna()]
datos_facturas_xgb = datos_facturas_xgb[datos_facturas_xgb['fecha_pago'].notna()]
datos_facturas_xgb.shape

Podemos ver como al aplicar la primera condición de nuestra estrategia nuestra muestra de facturas ha quedado reducida a 46410 facturas.

In [ ]:
datos_facturas_xgb['target'] = datos_facturas_xgb['fecha_pago'] - datos_facturas_xgb['fecha_documento']
datos_facturas_xgb['target'] = datos_facturas_xgb['target'].dt.days

In [ ]:
dates = dates.drop(['fecha_documento'], axis=1)
for i in dates:
    datos_facturas_xgb[i] = datos_facturas_xgb[i] - datos_facturas_xgb['fecha_documento']

In [ ]:
for i in dates:
    datos_facturas_xgb[i] = datos_facturas_xgb[i].dt.days

In [ ]:
for i in dates:
    datos_facturas_xgb.drop(datos_facturas_xgb[(datos_facturas_xgb[i] < 0)].index, inplace=True)
    print('El número de facturas sin error en la variable', i, 'o en anteriores columnas es:', len(datos_facturas_xgb))

In [ ]:
datos_facturas_xgb.shape

Confirmamos, como en el anterior modelo, que alrededor de 30000 facturas tienen su fecha de pago anterior a la fecha de emisión del documento y por lo tanto nos queda una muestra final para introducir al modelo de 14829 facturas.

A continuación veremos un gráfico que representa un histograma de lo que se tarda en pagar las facturas, que es lo que pretendemos predecir con este modelo.

In [ ]:
sns.histplot(data=datos_facturas_xgb, x='target')

In [ ]:
datos_facturas_xgb = datos_facturas_xgb.drop(['fecha_documento'], axis=1)

In [ ]:
datos_facturas_xgb.dtypes

In [ ]:
datos_facturas_xgb.head()

In [ ]:
datos_facturas_xgb.shape

El modelo contiene una muestra suficientemente grande, por lo que una vez terminada esta parte de prepararación del Dataset para el modelo vamos a empezar con su entrenamiento.

**Problemas con esta parte:**
- Muchas facturas con posibles errores en las fechas, entrenaremos al modelo con 14829 facturas, que respecto a las 46274 facturas que podrían ser si las fechas estuviesen correctas completamente.

[Volver al inicio del sección](#4)

[Volver al Índice](#contenido)

## 5. Entrenamiento del modelo<a id="5"></a>

En esta sección crearemos un regresor XGBoost con los parámetros por defecto.

In [ ]:
datos_facturas_xgb_1 = datos_facturas_xgb

In [ ]:
datos_facturas_xgb = datos_facturas_xgb.drop(['fecha_pago'], axis=1)

In [ ]:
X = datos_facturas_xgb.loc[:, datos_facturas_xgb.columns!='target']
y = datos_facturas_xgb.loc[:, 'target']
X.shape, y.shape, datos_facturas_xgb.shape

### train_test_split:
Separamos el conjunto de datos de la siguiente forma:
- 85% para entrenar
- 15% para test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.15, random_state=2000)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
xgbr = xgb.XGBRegressor(verbosity=0, seed=123)
classes_weights = class_weight.compute_sample_weight(class_weight='balanced', y=y_train)

In [ ]:
xgbr.fit(X_train, y_train, sample_weight=classes_weights)
preds1 = xgbr.predict(X_test)

A continuación veremos la importancia que le ha dado nuestro modelo XGBoost a cada variable, esto lo haremos mediante el siguiente proceso:
- **Feature Scores & Importance:** mide la ponderación y por lo tanto la importancia que el modelo le da a cada variable.

In [ ]:
feature_scores = pd.Series(xgbr.feature_importances_, index=X_train.columns).sort_values(ascending=False)
feature_scores

In [ ]:
feature_imp = pd.Series(xgbr.feature_importances_, index=X_train.columns.values).sort_values(ascending=False)

sns.barplot(x=feature_imp, y=feature_imp.index)
plt.xlabel('Feature Importance Score', fontsize=12)
plt.ylabel('Features', fontsize=12)
plt.title("Visualizing Feature Importances", fontsize=15)

In [ ]:
print(metrics.r2_score(y_test, preds1))

El $R^2$ o coeficiente de determinación es la métrica que hemos utilizado para medir el funcionamiento del modelo, este resultado que hemos obtenido nos quiere decir que el 99% de la varianza está explicada dentro del modelo.  
La varianza es la medida de dispersión que se utiliza para representar la variabilidad de un conjunto de datos respecto de la media aritmética de los mismos.

Para hablar más claro hemos calculado el RMSE (raíz del error cuadrático medio), de esta manera tenemos idea de la distancia media entre los datos observados y los datos que se han predicho. Nuestro resultado es 2.55, esto significa que la media de la diferencia entre lo que se tarda en pagar y los días que se han predicho que se iba a pagar son 2.55 días.

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, preds1))
print("RMSE : % f" %(rmse))

plt.figure(figsize=(10,10))
j = sns.regplot(x=y_test, y=preds1, fit_reg=True, scatter_kws={"s": 100})

La mayoría dentro de los pocos puntos que se alejan de nuestra recta de regresión se encuentran por debajo, esto significa que si se predijo que se iba a tardar 50 días en pagar la factura, las gran mayoría dentro de los que fallan tardan menos días en pagar por lo que tampoco supone un problema muy grave para Collecta.

In [ ]:
test_params = {
    "max_depth": [7],
    "learning_rate": [0.1],
    "gamma": [0.25],
    "reg_lambda": [0, 1],
    "scale_pos_weight": [1],
    "subsample": [0.8],
    "colsample_bytree": [0.5]
}

In [ ]:
model = GridSearchCV(estimator = xgbr, param_grid = test_params)
model.fit(X_train, y_train, sample_weight=classes_weights)
best_params = model.best_params_
print(best_params)
model.score(X_train, y_train)

In [ ]:
print("\n The best accuracy score across ALL searched parameters:\n", model.best_score_)

In [ ]:
xgbr_1 = xgb.XGBRegressor(verbosity=0, seed=123)
xgbr_1.set_params(**best_params)

In [ ]:
classes_weights = class_weight.compute_sample_weight(class_weight='balanced', y=y_train)
xgbr_1.fit(X_train, y_train, sample_weight=classes_weights)
preds2 = xgbr_1.predict(X_test)

In [ ]:
print(metrics.r2_score(y_test, preds2))

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, preds2))
print("RMSE : % f" %(rmse))

plt.figure(figsize=(10,10))
j = sns.regplot(x=y_test, y=preds2, fit_reg=True, scatter_kws={"s": 100})

**A continuación veremos otro modelo que será idéntico al que acabamos de ver salvo que no tendrá en cuenta las fechas ni la variable status.**

In [ ]:
datos_facturas_xgb = datos_facturas_xgb_1
datos_facturas_xgb = datos_facturas_xgb.drop(['fecha_recibida_aeesa', 'fecha_pago', 'fecha_reprogramacion',
                                             'fecha_revision', 'fecha_inserta'], axis=1)

datos_facturas_xgb = datos_facturas_xgb.drop(['status'], axis=1)

X_1 = datos_facturas_xgb.loc[:, datos_facturas_xgb.columns!='target']
y_1 = datos_facturas_xgb.loc[:, 'target']

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_1, y_1, test_size=.15, random_state=2000)

xgbr_2 = xgb.XGBRegressor(verbosity=0, seed=123)
classes_weights = class_weight.compute_sample_weight(class_weight='balanced', y=y_train_1)

xgbr_2.fit(X_train_1, y_train_1, sample_weight=classes_weights)
preds1 = xgbr_2.predict(X_test_1)

A continuación veremos la importancia que le ha dado nuestro modelo XGBoost a cada variable, esto lo haremos mediante el siguiente proceso:
- **Feature Scores & Importance:** mide la ponderación y por lo tanto la importancia que el modelo le da a cada variable.

In [ ]:
feature_scores = pd.Series(xgbr_2.feature_importances_, index=X_train_1.columns).sort_values(ascending=False)
feature_scores

In [ ]:
feature_imp = pd.Series(xgbr_2.feature_importances_, index=X_train_1.columns.values).sort_values(ascending=False)
sns.barplot(x=feature_imp, y=feature_imp.index)

plt.xlabel('Feature Importance Score', fontsize=12)
plt.ylabel('Features', fontsize=12)
plt.title("Visualizing Feature Importances", fontsize=15)

In [ ]:
print(metrics.r2_score(y_test_1, preds1))

rmse = np.sqrt(mean_squared_error(y_test_1, preds1))
print("RMSE : % f" %(rmse))

plt.figure(figsize=(10,10))
j = sns.regplot(x=y_test_1, y=preds1, fit_reg=True, scatter_kws={"s": 100})

In [ ]:
test_params = {
    "max_depth": [3, 4, 5, 7],
    "learning_rate": [0.1, 0.01, 0.05],
    "gamma": [0, 0.25, 0.75],
    "reg_lambda": [0, 1],
    "scale_pos_weight": [1, 3, 5],
    "subsample": [0.8],
    "colsample_bytree": [0.5]
}

model = GridSearchCV(estimator = xgbr_2, param_grid = test_params)
model.fit(X_train_1, y_train_1, sample_weight=classes_weights)
best_params = model.best_params_
print(best_params)
model.score(X_train_1, y_train_1)

print("\n The best accuracy score across ALL searched parameters:\n", model.best_score_)

In [ ]:
xgbr_3 = xgb.XGBRegressor(verbosity=0, seed=123)
xgbr_3.set_params(**best_params)

classes_weights = class_weight.compute_sample_weight(class_weight='balanced', y=y_train_1)
xgbr_3.fit(X_train_1, y_train_1, sample_weight=classes_weights)
preds2 = xgbr_3.predict(X_test_1)

print(metrics.r2_score(y_test_1, preds2))

rmse = np.sqrt(mean_squared_error(y_test_1, preds2))
print("RMSE : % f" %(rmse))

plt.figure(figsize=(10,10))
j = sns.regplot(x=y_test_1, y=preds2, fit_reg=True, scatter_kws={"s": 100})

[Volver al inicio del sección](#4)

[Volver al Índice](#contenido)

<a id="6"></a>
## 6. Ejemplo práctico

**Modelo con fechas y status:**

In [ ]:
for i in range(10):
    ind = y_test.sample(1).index.astype(int)
    print(y_test[ind])
    print(datos_facturas['fecha_documento'][ind])
    x_test = X.loc[ind]
    print(xgbr.predict(x_test), '\n')
    i += 1

**Modelo sin fechas y status:**

In [ ]:
for i in range(10):
    ind = y_test_1.sample(1).index.astype(int)
    print(y_test_1[ind])
    print(datos_facturas['fecha_documento'][ind])
    x_test_1 = X_1.loc[ind]
    print(xgbr_3.predict(x_test_1), '\n')
    i += 1

[Volver al inicio del sección](#6)

[Volver al Índice](#contenido)

<a id="7"></a>
## 7. Persistencia modelo

In [ ]:
dataset = pd.concat([X,y],axis=1)
dataset.to_csv('datasets/modelo_2.csv',index=False)
dataset.shape

In [ ]:
dataset.head()

In [ ]:
xgbr.save_model('models/Modelo_2.bst')